In [29]:
!pip install --upgrade gspread gspread_dataframe oauth2client

In [35]:
# Authenticate with Google in Colab
from google.colab import auth
#auth.authenticate_user()

# Authorize with gspread
import gspread
from google.auth import default

creds, _ = default()
gc = gspread.authorize(creds)

# Open Google Sheet by URL
sh = gc.open_by_url("https://docs.google.com/spreadsheets/d/1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8/edit#gid=1706329183")

worksheet = sh.worksheet("clap_general")

/tmp/ipython-input-35-3712123805.py:19: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C2', [[50.676]])


{'spreadsheetId': '1DG3jWi9nWZ2hEzOsO5t6xtNgsmDJ8sdKOmGxJZKkMl8',
 'updatedRange': 'clap_general!C2',
 'updatedRows': 1,
 'updatedColumns': 1,
 'updatedCells': 1}

In [1]:
from transformers import ClapModel, AutoProcessor
import torch
import librosa
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
# Load CLAP model + processor
model = ClapModel.from_pretrained("laion/larger_clap_general")
processor = AutoProcessor.from_pretrained("laion/larger_clap_general")

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/643 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/776M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/541 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

merges.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/280 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/776M [00:00<?, ?B/s]

In [3]:
print('total number of CLAP\'s parameters:', sum(p.numel() for p in model.parameters()))

total number of CLAP's parameters: 193913882


In [4]:
param_size = 0
for param in model.parameters():
    param_size += param.nelement() * param.element_size()
buffer_size = 0
for buffer in model.buffers():
    buffer_size += buffer.nelement() * buffer.element_size()

size_all_mb = (param_size + buffer_size) / 1024**2
print('CLAP model size: {:.3f}MB'.format(size_all_mb))

CLAP model size: 740.294MB


# Process audio

In [5]:
import zipfile
import os

zip_files = ["Exp1.zip", "Exp2.zip", "Analysis.zip"]

extract_dir = "/content/" # You can change this if you want to extract elsewhere
os.makedirs(extract_dir, exist_ok=True)

for zip_file in zip_files:
    if os.path.exists(zip_file):
        with zipfile.ZipFile(zip_file, 'r') as zip_ref:
            zip_ref.extractall(extract_dir)
        print(f"Extracted {zip_file} to {extract_dir}")
    else:
        print(f"Error: {zip_file} not found.")

Extracted Exp1.zip to /content/
Extracted Exp2.zip to /content/
Extracted Analysis.zip to /content/


In [61]:
audio_stimuli = []
stimuli_path = "/content/Exp1/Stimuli/"

for file in sorted(os.listdir(stimuli_path)):
    if file.endswith(".wav"):
        wav_path = os.path.join(stimuli_path, file)
        #Clap has already been trained on a sample rate of 48,000 so we should use what it knows already
        audio, sample_rate = librosa.load(wav_path, sr=48000)
        audio_stimuli.append(audio)

In [62]:
inputs = processor(audios=audio_stimuli, return_tensors="pt", padding=True, sampling_rate=48000)
audio_embeddings = model.get_audio_features(**inputs)

In [63]:
print(audio_embeddings.shape)
# Size of the 2^9 vector

torch.Size([59, 512])


# Process text

In [64]:
discrete_tags = ["happiness", "sadness", "anger", "tenderness", "fear"]
discrete_captions_perceived = [tag for tag in discrete_tags]
print(discrete_captions_perceived)

discrete_captions_induced = [tag for tag in discrete_tags]
print(discrete_captions_induced)

dimensional_tags = ["positive", "relaxed", "awake", "like"]
dimensional_captions_perceived = [tag for tag in dimensional_tags]
print(dimensional_captions_perceived)
dimensional_captions_induced = [tag for tag in dimensional_tags]
print(dimensional_captions_induced)


['happiness', 'sadness', 'anger', 'tenderness', 'fear']
['happiness', 'sadness', 'anger', 'tenderness', 'fear']
['positive', 'relaxed', 'awake', 'like']
['positive', 'relaxed', 'awake', 'like']


In [65]:
all_tags = discrete_captions_perceived + discrete_captions_induced + dimensional_captions_perceived + dimensional_captions_induced

# Using CLAP processor to tokenize (text-->numerical tokens) and prepare text captions
tag_inputs = processor(text=dimensional_captions_induced, return_tensors="pt", padding=True)
# Pass processed numerical tokens (from text) through claps text encoder --> text embeddings
tag_embeds = model.get_text_features(**tag_inputs)

In [66]:
tag_inputs_disc_i = processor(text=discrete_captions_induced, return_tensors="pt", padding=True)
tag_embeds_disc_i = model.get_text_features(**tag_inputs_disc_i)

In [67]:
tag_inputs_dim_p = processor(text=dimensional_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_dim_p = model.get_text_features(**tag_inputs_dim_p)

In [68]:
tag_inputs_disc_p = processor(text=discrete_captions_perceived, return_tensors="pt", padding=True)
tag_embeds_disc_p = model.get_text_features(**tag_inputs_disc_p)

# Generate outputs

In [69]:
# Similarity scores between audio and text embeddings (dot product)
sims = torch.matmul(audio_embeddings, tag_embeds.T)
print(sims.shape)

torch.Size([59, 4])


In [70]:
sims_disc_i = torch.matmul(audio_embeddings, tag_embeds_disc_i.T)
print(sims_disc_i.shape)

torch.Size([59, 5])


In [71]:
sims_dim_p = torch.matmul(audio_embeddings, tag_embeds_dim_p.T)
print(sims_dim_p.shape)

torch.Size([59, 4])


In [72]:
sims_disc_p = torch.matmul(audio_embeddings, tag_embeds_disc_p.T)
print(sims_disc_p.shape)

torch.Size([59, 5])


## Load csv files and extract related columns

In [73]:
IDim_path = '/content/Exp1/Data/IDim/'
#IDim: Indiced dimensional
IDim_responses = []

# All 65 responses concatenated to one data frame
all_dfs = []

for file in sorted(os.listdir(IDim_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake','like']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []
mean_like_vector = []


num_rows = all_dfs[0].shape[0]

# 59 rows
for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []
    current_row_likes = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])
        current_row_likes.append(df.iloc[i]['like'])


    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))
    mean_like_vector.append(np.mean(current_row_likes))


IDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector,
    'like_vector': mean_like_vector

}

# One value per stimulus (mean from all participants rating)
print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(IDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(IDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(IDim_responses['awake_vector'])}")
print(f"Length of like_vector: {len(IDim_responses['like_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59
Length of like_vector: 59


In [74]:
PDim_path = '/content/Exp1/Data/PDim/'
#PDim: Percieved dimensional
PDim_responses = []

all_dfs = []

for file in sorted(os.listdir(PDim_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(PDim_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ['positive', 'relaxed', 'awake','like']
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_positive_vector = []
mean_relaxed_vector = []
mean_awake_vector = []
mean_like_vector = []

num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_positives = []
    current_row_relaxeds = []
    current_row_awakes = []
    current_row_likes = []

    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_positives.append(df.iloc[i]['positive'])
        current_row_relaxeds.append(df.iloc[i]['relaxed'])
        current_row_awakes.append(df.iloc[i]['awake'])
        current_row_likes.append(df.iloc[i]['like'])

    # Calculate the mean for the current row across all files, for each column
    mean_positive_vector.append(np.mean(current_row_positives))
    mean_relaxed_vector.append(np.mean(current_row_relaxeds))
    mean_awake_vector.append(np.mean(current_row_awakes))
    mean_like_vector.append(np.mean(current_row_likes))

PDim_responses = {
    'positive_vector': mean_positive_vector,
    'relaxed_vector': mean_relaxed_vector,
    'awake_vector': mean_awake_vector,
    'like_vector': mean_like_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of positive_vector: {len(PDim_responses['positive_vector'])}")
print(f"Length of relaxed_vector: {len(PDim_responses['relaxed_vector'])}")
print(f"Length of awake_vector: {len(PDim_responses['awake_vector'])}")
print(f"Length of like_vector: {len(PDim_responses['like_vector'])}")


--- Mean Vectors (Mean across CSVs for each row position) ---
Length of positive_vector: 59
Length of relaxed_vector: 59
Length of awake_vector: 59
Length of like_vector: 59


In [75]:
IDisc_path = '/content/Exp1/Data/IDisc/'
#IDisc: Induced discrete
IDisc_responses = []

all_dfs = []

for file in sorted(os.listdir(IDisc_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(IDisc_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ["happiness", "sadness", "anger", "tenderness", "fear"]
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_happiness_vector = []
mean_sadness_vector = []
mean_anger_vector = []
mean_tenderness_vector = []
mean_fear_vector = []



num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_happiness = []
    current_row_sadness = []
    current_row_anger = []
    current_row_tenderness = []
    current_row_fear = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_happiness.append(df.iloc[i]['happiness'])
        current_row_sadness.append(df.iloc[i]['sadness'])
        current_row_anger.append(df.iloc[i]['anger'])
        current_row_tenderness.append(df.iloc[i]['tenderness'])
        current_row_fear.append(df.iloc[i]['fear'])


    # Calculate the mean for the current row across all files, for each column
    mean_happiness_vector.append(np.mean(current_row_happiness))
    mean_sadness_vector.append(np.mean(current_row_sadness))
    mean_anger_vector.append(np.mean(current_row_anger))
    mean_tenderness_vector.append(np.mean(current_row_tenderness))
    mean_fear_vector.append(np.mean(current_row_fear))

IDisc_responses = {
    'happiness_vector': mean_happiness_vector,
    'sadness_vector': mean_sadness_vector,
    'anger_vector': mean_anger_vector,
    'tenderness_vector': mean_tenderness_vector,
    'fear_vector': mean_fear_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of happiness_vector: {len(IDisc_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(IDisc_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(IDisc_responses['anger_vector'])}")
print(f"Length of tenderness_vector: {len(IDisc_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(IDisc_responses['fear_vector'])}")



--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tenderness_vector: 59
Length of fear_vector: 59


In [76]:
PDisc_path = '/content/Exp1/Data/PDisc/'
#PDisc: Percieved discrete
PDisc_responses = []

all_dfs = []

for file in sorted(os.listdir(PDisc_path)):
    if file.endswith(".csv"):
        df = pd.read_csv(os.path.join(PDisc_path, file))
        df.columns = df.columns.str.strip()
        required_columns = ["happiness", "sadness", "anger", "tenderness", "fear"]
        if all(col in df.columns for col in required_columns):
            all_dfs.append(df)

mean_happiness_vector = []
mean_sadness_vector = []
mean_anger_vector = []
mean_tenderness_vector = []
mean_fear_vector = []



num_rows = all_dfs[0].shape[0]

for i in range(num_rows):
    current_row_happiness = []
    current_row_sadness = []
    current_row_anger = []
    current_row_tenderness = []
    current_row_fear = []


    # For the current row index 'i', collect values from all DataFrames
    for df in all_dfs:
        current_row_happiness.append(df.iloc[i]['happiness'])
        current_row_sadness.append(df.iloc[i]['sadness'])
        current_row_anger.append(df.iloc[i]['anger'])
        current_row_tenderness.append(df.iloc[i]['tenderness'])
        current_row_fear.append(df.iloc[i]['fear'])


    # Calculate the mean for the current row across all files, for each column
    mean_happiness_vector.append(np.mean(current_row_happiness))
    mean_sadness_vector.append(np.mean(current_row_sadness))
    mean_anger_vector.append(np.mean(current_row_anger))
    mean_tenderness_vector.append(np.mean(current_row_tenderness))
    mean_fear_vector.append(np.mean(current_row_fear))

PDisc_responses = {
    'happiness_vector': mean_happiness_vector,
    'sadness_vector': mean_sadness_vector,
    'anger_vector': mean_anger_vector,
    'tenderness_vector': mean_tenderness_vector,
    'fear_vector': mean_fear_vector
}

print("\n--- Mean Vectors (Mean across CSVs for each row position) ---")
print(f"Length of happiness_vector: {len(PDisc_responses['happiness_vector'])}")
print(f"Length of sadness_vector: {len(PDisc_responses['sadness_vector'])}")
print(f"Length of anger_vector: {len(PDisc_responses['anger_vector'])}")
print(f"Length of tenderness_vector: {len(PDisc_responses['tenderness_vector'])}")
print(f"Length of fear_vector: {len(PDisc_responses['fear_vector'])}")



--- Mean Vectors (Mean across CSVs for each row position) ---
Length of happiness_vector: 59
Length of sadness_vector: 59
Length of anger_vector: 59
Length of tenderness_vector: 59
Length of fear_vector: 59


# Evaluate

In [77]:
scaled_sims = torch.tensor(sims)

for i in range(sims.shape[1]):
    sims_column = sims[:,i]
    # Find the min and max values in the current sims tensor
    old_min = sims_column.min()
    old_max = sims_column.max()

    # Define the new desired range
    new_min = 1.0
    new_max = 9.0

    # Apply the min-max scaling formula
    scaled_sims[:,i] = ((sims_column - old_min) / (old_max - old_min)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims[:10]}\n")
print(f"Scaled sims min value: {scaled_sims.min():.4f}")
print(f"Scaled sims max value: {scaled_sims.max():.4f}\n")

human_ratings_tensor = torch.tensor([
    IDim_responses['positive_vector'],
    IDim_responses['relaxed_vector'],
    IDim_responses['awake_vector'],
    IDim_responses['like_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor[:10]}\n")


Scaled sims shape: torch.Size([59, 4])
Scaled sims (first 5 rows):
tensor([[2.6545, 5.5930, 6.8049, 6.6757],
        [5.2898, 6.5823, 7.1383, 7.1481],
        [3.3807, 6.5599, 5.6638, 5.6381],
        [1.0000, 6.0628, 4.4661, 5.3958],
        [5.0035, 2.6858, 6.5212, 6.0008],
        [4.0618, 4.1813, 6.1806, 4.6919],
        [3.4975, 5.9934, 6.4437, 6.1349],
        [5.6666, 6.9776, 8.4851, 8.1195],
        [2.5932, 4.6727, 5.8753, 4.2798],
        [4.0595, 6.7544, 6.3593, 6.1079]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 4])
Human ratings tensor (first 5 rows):
tensor([[4.4872, 4.8798, 4.5185, 4.6463],
        [4.7682, 5.3057, 4.7414, 5.0326],
        [5.3105, 5.7312, 5.0760, 5.3854],
        [5.2582, 5.2651, 5.4488, 5.2492],
        [3.8897, 3.9540, 5.3549, 3.7985],
        [4.3197, 3.8900, 5.5089, 3.7712],
        [4.9825, 5.1875, 5.5354, 4.9312],
        [5.0934, 4.8657, 5.8208, 4.7805],
   

/tmp/ipython-input-77-2618908339.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scaled_sims = torch.tensor(sims)


In [78]:
scaled_sims_disc_i = torch.tensor(sims_disc_i)

for i in range(sims_disc_i.shape[1]):
    sims_column_disc_i = sims_disc_i[:,i]
    # Find the min and max values in the current sims tensor
    old_min_disc_i = sims_column_disc_i.min()
    old_max_disc_i = sims_column_disc_i.max()

    # Define the new desired range
    new_min = 1.0
    new_max = 9.0

    # Apply the min-max scaling formula
    scaled_sims_disc_i[:,i] = ((sims_column_disc_i - old_min_disc_i) / (old_max_disc_i - old_min_disc_i)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_disc_i.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_disc_i[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_disc_i.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_disc_i.max():.4f}\n")

human_ratings_tensor_disc_i = torch.tensor([
    IDisc_responses['happiness_vector'],
    IDisc_responses['sadness_vector'],
    IDisc_responses['anger_vector'],
    IDisc_responses['tenderness_vector'],
    IDisc_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor_disc_i.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_disc_i[:10]}\n")

Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[5.0214, 6.6733, 6.8325, 5.7862, 7.7550],
        [3.9333, 4.9207, 7.4465, 4.9929, 7.3572],
        [3.4530, 4.3210, 6.1135, 4.4504, 6.8921],
        [3.3673, 3.4643, 4.7160, 4.4292, 5.9314],
        [4.9545, 4.8828, 5.6719, 5.3675, 6.6845],
        [3.4116, 3.7599, 5.4187, 4.8446, 5.3570],
        [3.1252, 4.5116, 7.3265, 4.9726, 6.9538],
        [4.7480, 6.7702, 8.3918, 6.6676, 8.6063],
        [3.8313, 4.5940, 5.1478, 5.3246, 5.6200],
        [4.4003, 6.8146, 6.6961, 6.2695, 7.7511]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.8202, 3.3418, 2.7948, 1.9789, 4.1430],
        [1.9091, 3.4459, 2.9059, 2.3785, 2.8558],
        [2.0644, 3.1847, 2.4182, 2.6371, 2.3356],
        [2.5321, 3.1900, 2.1509, 3.0583, 2.0938],
        [1.8391, 2.4733, 4.0144, 1.9902, 3.4942],
        [

/tmp/ipython-input-78-333891882.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scaled_sims_disc_i = torch.tensor(sims_disc_i)


In [79]:
scaled_sims_disc_p = torch.tensor(sims_disc_p)

for i in range(sims_disc_p.shape[1]):
    sims_column_disc_p = sims_disc_p[:,i]
    # Find the min and max values in the current sims tensor
    old_min_disc_p = sims_column_disc_p.min()
    old_max_disc_p = sims_column_disc_p.max()

    # Define the new desired range
    new_min = 1.0
    new_max = 9.0

    # Apply the min-max scaling formula
    scaled_sims_disc_p[:,i] = ((sims_column_disc_p - old_min_disc_p) / (old_max_disc_p - old_min_disc_p)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_disc_p.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_disc_p[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_disc_p.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_disc_p.max():.4f}\n")

human_ratings_tensor_disc_p = torch.tensor([
    PDisc_responses['happiness_vector'],
    PDisc_responses['sadness_vector'],
    PDisc_responses['anger_vector'],
    PDisc_responses['tenderness_vector'],
    PDisc_responses['fear_vector']
], dtype=torch.float32).T # Transpose to get shape (59, 5)

print(f"Human ratings tensor shape: {human_ratings_tensor_disc_p.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_disc_p[:10]}\n")

Scaled sims shape: torch.Size([59, 5])
Scaled sims (first 5 rows):
tensor([[5.0214, 6.6733, 6.8325, 5.7862, 7.7550],
        [3.9333, 4.9207, 7.4465, 4.9929, 7.3572],
        [3.4530, 4.3210, 6.1135, 4.4504, 6.8921],
        [3.3673, 3.4643, 4.7160, 4.4292, 5.9314],
        [4.9545, 4.8828, 5.6719, 5.3675, 6.6845],
        [3.4116, 3.7599, 5.4187, 4.8446, 5.3570],
        [3.1252, 4.5116, 7.3265, 4.9726, 6.9538],
        [4.7480, 6.7702, 8.3918, 6.6676, 8.6063],
        [3.8313, 4.5940, 5.1478, 5.3246, 5.6200],
        [4.4003, 6.8146, 6.6961, 6.2695, 7.7511]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 5])
Human ratings tensor (first 5 rows):
tensor([[1.5362, 4.4191, 5.1551, 2.0345, 5.6468],
        [1.7403, 4.8642, 4.2637, 2.5285, 3.9835],
        [2.0486, 5.5669, 3.0558, 2.9492, 2.8565],
        [2.4389, 4.8343, 2.1514, 3.5546, 3.0082],
        [1.5691, 3.6631, 5.0478, 1.9322, 4.7326],
        [

/tmp/ipython-input-79-4230085643.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scaled_sims_disc_p = torch.tensor(sims_disc_p)


In [80]:
scaled_sims_dim_p = torch.tensor(sims_dim_p)

for i in range(sims_dim_p.shape[1]):
    sims_column_dim_p = sims_dim_p[:,i]
    # Find the min and max values in the current sims tensor
    old_min_dim_p = sims_column_dim_p.min()
    old_max_dim_p = sims_column_dim_p.max()

    # Define the new desired range
    new_min = 1.0
    new_max = 9.0

    # Apply the min-max scaling formula
    scaled_sims_dim_p[:,i] = ((sims_column_dim_p - old_min_dim_p) / (old_max_dim_p - old_min_dim_p)) * (new_max - new_min) + new_min

print(f"Scaled sims shape: {scaled_sims_dim_p.shape}")
print(f"Scaled sims (first 5 rows):\n{scaled_sims_dim_p[:10]}\n")
print(f"Scaled sims min value: {scaled_sims_dim_p.min():.4f}")
print(f"Scaled sims max value: {scaled_sims_dim_p.max():.4f}\n")

human_ratings_tensor_dim_p = torch.tensor([
    PDim_responses['positive_vector'],
    PDim_responses['relaxed_vector'],
    PDim_responses['awake_vector'],
    PDim_responses['like_vector'],
], dtype=torch.float32).T # Transpose to get shape (59, 3)

print(f"Human ratings tensor shape: {human_ratings_tensor_dim_p.shape}")
print(f"Human ratings tensor (first 5 rows):\n{human_ratings_tensor_dim_p[:10]}\n")

Scaled sims shape: torch.Size([59, 4])
Scaled sims (first 5 rows):
tensor([[2.6545, 5.5930, 6.8049, 6.6757],
        [5.2898, 6.5823, 7.1383, 7.1481],
        [3.3807, 6.5599, 5.6638, 5.6381],
        [1.0000, 6.0628, 4.4661, 5.3958],
        [5.0035, 2.6858, 6.5212, 6.0008],
        [4.0618, 4.1813, 6.1806, 4.6919],
        [3.4975, 5.9934, 6.4437, 6.1349],
        [5.6666, 6.9776, 8.4851, 8.1195],
        [2.5932, 4.6727, 5.8753, 4.2798],
        [4.0595, 6.7544, 6.3593, 6.1079]], grad_fn=<SliceBackward0>)

Scaled sims min value: 1.0000
Scaled sims max value: 9.0000

Human ratings tensor shape: torch.Size([59, 4])
Human ratings tensor (first 5 rows):
tensor([[3.5116, 4.4973, 3.4563, 4.7657],
        [4.1519, 4.9952, 3.2785, 5.1364],
        [4.6982, 5.4154, 4.2416, 5.2006],
        [5.2018, 5.2612, 4.9213, 4.9566],
        [3.1964, 3.4387, 3.9169, 3.3366],
        [3.9328, 3.8416, 4.5261, 3.9254],
        [4.5933, 4.8528, 4.9660, 4.4846],
        [4.5391, 4.6797, 5.4388, 4.7018],
   

/tmp/ipython-input-80-1306616557.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  scaled_sims_dim_p = torch.tensor(sims_dim_p)


 # Comparison Metrics

In [81]:
from sklearn.metrics import mean_absolute_error, mean_squared_error
from scipy.stats import pearsonr # For Pearson correlation

# Comparison Method 1: Mean Absolute Error (MAE)
# MAE measures the average magnitude of the errors in a set of predictions, without considering their direction.
# Average of all absolute errors across all 59 stimuli and 3 dimensions.
# e.g An MAE of 1.5 means CLAP is typically ±1.5 points away from human ratings.
# Mean is calculated over the matrix so one value is outputted
mae = torch.mean(torch.abs(scaled_sims - human_ratings_tensor))
worksheet.update('C2', [[round(mae.item(), 4)]])

mae_disc_i = torch.mean(torch.abs(scaled_sims_disc_i - human_ratings_tensor_disc_i))
worksheet.update('C16', [[round(mae_disc_i.item(), 4)]])

mae_disc_p = torch.mean(torch.abs(scaled_sims_disc_p - human_ratings_tensor_disc_p))
worksheet.update('C23', [[round(mae_disc_p.item(), 4)]])

mae_dim_p = torch.mean(torch.abs(scaled_sims_dim_p - human_ratings_tensor_dim_p))
worksheet.update('C9', [[round(mae_dim_p.item(), 4)]])

print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDim) : {mae:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDisc) : {mae_disc_i:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDisc) : {mae_disc_p:.4f}")
print(f"Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDim) : {mae_dim_p:.4f}")

# Comparison Method 2: Pearson Correlation Coefficient (per column)
# Pearson correlation measures the linear relationship between two sets of data.
# We'll calculate it for each of the three columns (dimensions).
# Which emotion dimensions does CLAP understand best?

#IDim
correlation_positive, _ = pearsonr(scaled_sims[:, 0].detach().numpy(), human_ratings_tensor[:, 0].numpy())
correlation_relaxed, _ = pearsonr(scaled_sims[:, 1].detach().numpy(), human_ratings_tensor[:, 1].numpy())
correlation_awake, _ = pearsonr(scaled_sims[:, 2].detach().numpy(), human_ratings_tensor[:, 2].numpy())
correlation_like, _ = pearsonr(scaled_sims[:, 3].detach().numpy(), human_ratings_tensor[:, 3].numpy())

worksheet.update(
    range_name='C3',
    values=[[f"{correlation_positive:.4f}, {correlation_relaxed:.4f}, {correlation_awake:.4f}, {correlation_like:.4f}"]]
)

#PDim
correlation_positive_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 0].detach().numpy(), human_ratings_tensor_dim_p[:, 0].numpy())
correlation_relaxed_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 1].detach().numpy(), human_ratings_tensor_dim_p[:, 1].numpy())
correlation_awake_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 2].detach().numpy(), human_ratings_tensor_dim_p[:, 2].numpy())
correlation_like_dim_p, _ = pearsonr(scaled_sims_dim_p[:, 3].detach().numpy(), human_ratings_tensor_dim_p[:, 3].numpy())

worksheet.update(
    range_name='C10',
    values=[[f"{correlation_positive_dim_p:.4f}, {correlation_relaxed_dim_p:.4f}, {correlation_awake_dim_p:.4f}, {correlation_like_dim_p:.4f}"]]
)

# IDisc
correlation_happiness, _ = pearsonr(scaled_sims_disc_i[:, 0].detach().numpy(), human_ratings_tensor_disc_i[:, 0].numpy())
correlation_sadness, _ = pearsonr(scaled_sims_disc_i[:, 1].detach().numpy(), human_ratings_tensor_disc_i[:, 1].numpy())
correlation_anger, _ = pearsonr(scaled_sims_disc_i[:, 2].detach().numpy(), human_ratings_tensor_disc_i[:, 2].numpy())
correlation_tenderness, _ = pearsonr(scaled_sims_disc_i[:, 3].detach().numpy(), human_ratings_tensor_disc_i[:, 3].numpy())
correlation_fear, _ = pearsonr(scaled_sims_disc_i[:, 4].detach().numpy(), human_ratings_tensor_disc_i[:, 4].numpy())

worksheet.update(
    range_name='C17',
    values=[[f"{correlation_happiness:.4f}, {correlation_sadness:.4f}, {correlation_anger:.4f}, {correlation_tenderness:.4f}, {correlation_fear:.4f}"]]
)

#PDisc
correlation_happiness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 0].detach().numpy(), human_ratings_tensor_disc_p[:, 0].numpy())
correlation_sadness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 1].detach().numpy(), human_ratings_tensor_disc_p[:, 1].numpy())
correlation_anger_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 2].detach().numpy(), human_ratings_tensor_disc_p[:, 2].numpy())
correlation_tenderness_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 3].detach().numpy(), human_ratings_tensor_disc_p[:, 3].numpy())
correlation_fear_disc_p, _ = pearsonr(scaled_sims_disc_p[:, 4].detach().numpy(), human_ratings_tensor_disc_p[:, 4].numpy())

worksheet.update(
    range_name='C24',
    values=[[f"{correlation_happiness_disc_p:.4f}, {correlation_sadness_disc_p:.4f}, {correlation_anger_disc_p:.4f}, {correlation_tenderness_disc_p:.4f}, {correlation_fear_disc_p:.4f}"]]
)

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDim:")
print(f"  Positive Dimension: {correlation_positive:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed:.4f}")
print(f"  Awake Dimension:    {correlation_awake:.4f}\n")
print(f"  Like Dimension:    {correlation_like:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDim:")
print(f"  Positive Dimension: {correlation_positive_dim_p:.4f}")
print(f"  Relaxed Dimension:  {correlation_relaxed_dim_p:.4f}")
print(f"  Awake Dimension:    {correlation_awake_dim_p:.4f}\n")
print(f"  Like Dimension:    {correlation_like_dim_p:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDisc:")
print(f"  Happiness Dimension: {correlation_happiness:.4f}")
print(f"  Sadness Dimension:  {correlation_sadness:.4f}")
print(f"  Anger Dimension:    {correlation_anger:.4f}")
print(f"  Tenderness Dimension:    {correlation_tenderness:.4f}")
print(f"  Fear Dimension:    {correlation_fear:.4f}\n")

print("Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDisc:")
print(f"  Happiness Dimension: {correlation_happiness_disc_p:.4f}")
print(f"  Sadness Dimension:  {correlation_sadness_disc_p:.4f}")
print(f"  Anger Dimension:    {correlation_anger_disc_p:.4f}")
print(f"  Tenderness Dimension:    {correlation_tenderness_disc_p:.4f}")
print(f"  Fear Dimension:    {correlation_fear_disc_p:.4f}\n")

average_correlation = (correlation_positive + correlation_relaxed + correlation_awake + correlation_like ) / 4
worksheet.update(
    range_name='C4',
    values=[[f"{average_correlation:.4f}"]]
)

average_correlation_disc_i = (correlation_happiness + correlation_sadness + correlation_anger + correlation_tenderness + correlation_fear) / 5
worksheet.update(
    range_name='C18',
    values=[[f"{average_correlation_disc_i:.4f}"]]
)

average_correlation_disc_p = (correlation_happiness_disc_p + correlation_sadness_disc_p + correlation_anger_disc_p + correlation_tenderness_disc_p + correlation_fear_disc_p) / 5
worksheet.update(
    range_name='C25',
    values=[[f"{average_correlation_disc_p:.4f}"]]
)

average_correlation_dim_p = (correlation_positive_dim_p + correlation_relaxed_dim_p + correlation_awake_dim_p + correlation_like_dim_p) / 4
worksheet.update(
    range_name='C11',
    values=[[f"{average_correlation_dim_p:.4f}"]]
)

print(f"  Average Correlation (IDim): {average_correlation:.4f}")
print(f"  Average Correlation (IDisc): {average_correlation_disc_i:.4f}")
print(f"  Average Correlation (PDisc): {average_correlation_disc_p:.4f}")
print(f"  Average Correlation (PDim): {average_correlation_dim_p:.4f}")

/tmp/ipython-input-81-258984054.py:10: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C2', [[round(mae.item(), 4)]])
/tmp/ipython-input-81-258984054.py:13: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C16', [[round(mae_disc_i.item(), 4)]])
/tmp/ipython-input-81-258984054.py:16: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C23', [[round(mae_disc_p.item(), 4)]])
/tmp/ipython-input-81-258984054.py:19: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, 

Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDim) : 1.7904
Mean Absolute Error (MAE) between scaled_sims and human_ratings (IDisc) : 2.3119
Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDisc) : 1.9962
Mean Absolute Error (MAE) between scaled_sims and human_ratings (PDim) : 1.7941
Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDim:
  Positive Dimension: 0.2607
  Relaxed Dimension:  0.2174
  Awake Dimension:    -0.2840

  Like Dimension:    -0.3913

Pearson Correlation Coefficients (between scaled_sims and human_ratings) PDim:
  Positive Dimension: 0.4724
  Relaxed Dimension:  0.2103
  Awake Dimension:    -0.3106

  Like Dimension:    -0.4483

Pearson Correlation Coefficients (between scaled_sims and human_ratings) IDisc:
  Happiness Dimension: 0.4121
  Sadness Dimension:  -0.1928
  Anger Dimension:    0.3405
  Tenderness Dimension:    0.2274
  Fear Dimension:    0.2192

Pearson Correlation Coefficients (between scaled_s

In [82]:
# Comparison Method 2: Mean Absolute Percentage Error (MAPE)
# MAPE measures the accuracy of a forecasting method in terms of percentage.
# Formula: MAPE = (1/n) * sum(|(Actual - Forecast) / Actual|) * 100%
# On average, what percentage do CLAP's predictions deviate from human ratings?

# Calculate the absolute percentage error for each element
# Since human responses are between 1 and 9, division by zero is not a concern.
absolute_percentage_error = torch.abs((human_ratings_tensor - scaled_sims) / human_ratings_tensor) * 100
absolute_percentage_error_disc_i = torch.abs((human_ratings_tensor_disc_i - scaled_sims_disc_i) / human_ratings_tensor_disc_i) * 100
absolute_percentage_error_disc_p = torch.abs((human_ratings_tensor_disc_p - scaled_sims_disc_p) / human_ratings_tensor_disc_p) * 100
absolute_percentage_error_dim_p = torch.abs((human_ratings_tensor_dim_p - scaled_sims_dim_p) / human_ratings_tensor_dim_p) * 100

# Calculate the mean of these percentage errors
mape = torch.mean(absolute_percentage_error)
worksheet.update('C5', [[f"{mape:.2f}%"]])

mape_disc_i = torch.mean(absolute_percentage_error_disc_i)
worksheet.update('C19', [[f"{mape_disc_i:.2f}%"]])

mape_disc_p = torch.mean(absolute_percentage_error_disc_p)
worksheet.update('C26', [[f"{mape_disc_p:.2f}%"]])

mape_dim_p = torch.mean(absolute_percentage_error_dim_p)
worksheet.update('C12', [[f"{mape_dim_p:.2f}%"]])


print(f"Mean Absolute Percentage Error (MAPE) IDim: {mape:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) IDisc: {mape_disc_i:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) PDisc: {mape_disc_p:.2f}%")
print(f"Mean Absolute Percentage Error (MAPE) PDim: {mape_dim_p:.2f}%\n")

# Comparison Method 3: Root Mean Squared Error (RMSE)
# RMSE measures the square root of the average of the squared differences between predicted and actual values.
# It gives a relatively high weight to large errors.
# Formula: RMSE = sqrt(mean((Actual - Forecast)^2))
# How far, on average, are CLAP’s predictions from human ratings—while heavily punishing big errors?

# Calculate the squared differences
squared_differences = (human_ratings_tensor - scaled_sims)**2
squared_differences_disc_i = (human_ratings_tensor_disc_i - scaled_sims_disc_i)**2
squared_differences_disc_p = (human_ratings_tensor_disc_p - scaled_sims_disc_p)**2
squared_differences_dim_p = (human_ratings_tensor_dim_p - scaled_sims_dim_p)**2


# Calculate the mean of the squared differences (Mean Squared Error - MSE)
mse = torch.mean(squared_differences)
mse_disc_i = torch.mean(squared_differences_disc_i)
mse_disc_p = torch.mean(squared_differences_disc_p)
mse_dim_p = torch.mean(squared_differences_dim_p)

# Calculate the square root to get RMSE
rmse = torch.sqrt(mse)
worksheet.update('C6', [[f"{rmse:.4f}"]])

rmse_disc_i = torch.sqrt(mse_disc_i)
worksheet.update('C20', [[f"{rmse_disc_i:.4f}"]])

rmse_disc_p = torch.sqrt(mse_disc_p)
worksheet.update('C27', [[f"{rmse_disc_p:.4f}"]])

rmse_dim_p = torch.sqrt(mse_dim_p)
worksheet.update('C13', [[f"{rmse_dim_p:.4f}"]])


print(f"Root Mean Squared Error (RMSE) IDim: {rmse:.4f}\n")
print(f"Root Mean Squared Error (RMSE) IDisc: {rmse_disc_i:.4f}\n")
print(f"Root Mean Squared Error (RMSE) PDisc: {rmse_disc_p:.4f}\n")
print(f"Root Mean Squared Error (RMSE) PDim: {rmse_dim_p:.4f}\n")

from sklearn.metrics import r2_score

# Comparison Method 4: R-Squared
# Measures how well CLAP's predictions explain the variance in human ratings.
# What percentage of the changes in human ratings can be predicted by CLAP's model? (1=perfect prediction)

# R-squared IDim
print("\nR-squared scores (IDim):")
r2_valence = r2_score(human_ratings_tensor[:, 0].numpy(), scaled_sims[:, 0].detach().numpy())
print("  valence =", r2_valence)
r2_tension = r2_score(human_ratings_tensor[:, 1].numpy(), scaled_sims[:, 1].detach().numpy())
print("  tension =", r2_tension)
r2_energy = r2_score(human_ratings_tensor[:, 2].numpy(), scaled_sims[:, 2].detach().numpy())
print("  energy =", r2_energy)
r2_like = r2_score(human_ratings_tensor[:, 3].numpy(), scaled_sims[:, 3].detach().numpy())
print("  like =", r2_like)

worksheet.update(
    range_name='C7',
    values=[[f"{r2_valence:.4f}, {r2_tension:.4f}, {r2_energy:.4f}, {r2_like:.4f}"]]
)

# R-squared IDisc
print("\nR-squared scores (IDisc):")
r2_happiness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 0].numpy(), scaled_sims_disc_i[:, 0].detach().numpy())
print("  happiness =", r2_happiness_disc_i)
r2_sadness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 1].numpy(), scaled_sims_disc_i[:, 1].detach().numpy())
print("  sadness =", r2_sadness_disc_i)
r2_anger_disc_i = r2_score(human_ratings_tensor_disc_i[:, 2].numpy(), scaled_sims_disc_i[:, 2].detach().numpy())
print("  anger =", r2_anger_disc_i)
r2_tenderness_disc_i = r2_score(human_ratings_tensor_disc_i[:, 3].numpy(), scaled_sims_disc_i[:, 3].detach().numpy())
print("  tenderness =", r2_tenderness_disc_i)
r2_fear_disc_i = r2_score(human_ratings_tensor_disc_i[:, 4].numpy(), scaled_sims_disc_i[:, 4].detach().numpy())
print("  fear =", r2_fear_disc_i)

worksheet.update(
    range_name='C21',
    values=[[f"{r2_happiness_disc_i:.4f}, {r2_sadness_disc_i:.4f}, {r2_anger_disc_i:.4f}, {r2_tenderness_disc_i:.4f}, {r2_fear_disc_i:.4f}"]]
)

# R-squared PDim
print("\nR-squared scores (PDim):")
r2_valence_dim_p = r2_score(human_ratings_tensor_dim_p[:, 0].numpy(), scaled_sims_dim_p[:, 0].detach().numpy())
print("  valence =", r2_valence_dim_p)
r2_tension_dim_p = r2_score(human_ratings_tensor_dim_p[:, 1].numpy(), scaled_sims_dim_p[:, 1].detach().numpy())
print("  tension =", r2_tension_dim_p)
r2_energy_dim_p = r2_score(human_ratings_tensor_dim_p[:, 2].numpy(), scaled_sims_dim_p[:, 2].detach().numpy())
print("  energy =", r2_energy_dim_p)
r2_like_dim_p = r2_score(human_ratings_tensor_dim_p[:, 3].numpy(), scaled_sims_dim_p[:, 3].detach().numpy())
print("  like =", r2_like_dim_p)

worksheet.update(
    range_name='C14',
    values=[[f"{r2_valence_dim_p:.4f}, {r2_tension_dim_p:.4f}, {r2_energy_dim_p:.4f}, {r2_like_dim_p:.4f}"]]
)

# R-squared PDisc
print("\nR-squared scores (PDisc):")
r2_happiness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 0].numpy(), scaled_sims_disc_p[:, 0].detach().numpy())
print("  happiness =", r2_happiness_disc_p)
r2_sadness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 1].numpy(), scaled_sims_disc_p[:, 1].detach().numpy())
print("  sadness =", r2_sadness_disc_p)
r2_anger_disc_p = r2_score(human_ratings_tensor_disc_p[:, 2].numpy(), scaled_sims_disc_p[:, 2].detach().numpy())
print("  anger =", r2_anger_disc_p)
r2_tenderness_disc_p = r2_score(human_ratings_tensor_disc_p[:, 3].numpy(), scaled_sims_disc_p[:, 3].detach().numpy())
print("  tenderness =", r2_tenderness_disc_p)
r2_fear_disc_p = r2_score(human_ratings_tensor_disc_p[:, 4].numpy(), scaled_sims_disc_p[:, 4].detach().numpy())
print("  fear =", r2_fear_disc_p)

worksheet.update(
    range_name='C28',
    values=[[f"{r2_happiness_disc_p:.4f}, {r2_sadness_disc_p:.4f}, {r2_anger_disc_p:.4f}, {r2_tenderness_disc_p:.4f}, {r2_fear_disc_p:.4f}"]]
)

average_r2 = (r2_valence + r2_energy + r2_tension + r2_like) /4
worksheet.update('C8', [[f"{average_r2:.4f}"]])

average_r2_dim_p = (r2_valence_dim_p + r2_energy_dim_p + r2_tension_dim_p + r2_like_dim_p) /4
worksheet.update('C15', [[f"{average_r2_dim_p:.4f}"]])

average_r2_disc_i = (r2_happiness_disc_i + r2_sadness_disc_i + r2_anger_disc_i + r2_tenderness_disc_i + r2_fear_disc_i) / 5
worksheet.update('C22', [[f"{average_r2_disc_i:.4f}"]])

average_r2_disc_p = (r2_happiness_disc_p + r2_sadness_disc_p + r2_anger_disc_p + r2_tenderness_disc_p + r2_fear_disc_p) / 5
worksheet.update('C29', [[f"{average_r2_disc_p:.4f}"]])

print("\nAverage R-squared scores:")
print("IDim: ", average_r2)
print("PDim: ", average_r2_dim_p)
print("IDisc: ", average_r2_disc_i)
print("PDisc: ", average_r2_disc_p)


/tmp/ipython-input-82-3604228928.py:15: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C5', [[f"{mape:.2f}%"]])
/tmp/ipython-input-82-3604228928.py:18: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C19', [[f"{mape_disc_i:.2f}%"]])
/tmp/ipython-input-82-3604228928.py:21: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C26', [[f"{mape_disc_p:.2f}%"]])
/tmp/ipython-input-82-3604228928.py:24: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  wor

Mean Absolute Percentage Error (MAPE) IDim: 36.60%
Mean Absolute Percentage Error (MAPE) IDisc: 99.56%
Mean Absolute Percentage Error (MAPE) PDisc: 75.05%
Mean Absolute Percentage Error (MAPE) PDim: 35.92%



/tmp/ipython-input-82-3604228928.py:53: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C6', [[f"{rmse:.4f}"]])
/tmp/ipython-input-82-3604228928.py:56: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C20', [[f"{rmse_disc_i:.4f}"]])
/tmp/ipython-input-82-3604228928.py:59: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C27', [[f"{rmse_disc_p:.4f}"]])
/tmp/ipython-input-82-3604228928.py:62: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksh

Root Mean Squared Error (RMSE) IDim: 2.2153

Root Mean Squared Error (RMSE) IDisc: 2.8144

Root Mean Squared Error (RMSE) PDisc: 2.4813

Root Mean Squared Error (RMSE) PDim: 2.2302


R-squared scores (IDim):
  valence = -1.6274635791778564
  tension = -1.6950757503509521
  energy = -16.466978073120117
  like = -2.7606329917907715

R-squared scores (IDisc):
  happiness = -4.228033065795898
  sadness = -26.900184631347656
  anger = -5.930661201477051
  tenderness = -13.123933792114258
  fear = -14.338150024414062

R-squared scores (PDim):
  valence = -0.7602062225341797
  tension = -1.8176486492156982
  energy = -5.768449783325195
  like = -2.8689684867858887

R-squared scores (PDisc):
  happiness = -2.2536935806274414
  sadness = -6.38144063949585
  anger = -3.806398391723633
  tenderness = -4.605845928192139
  fear = -4.3067731857299805


/tmp/ipython-input-82-3604228928.py:145: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C8', [[f"{average_r2:.4f}"]])
/tmp/ipython-input-82-3604228928.py:148: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C15', [[f"{average_r2_dim_p:.4f}"]])
/tmp/ipython-input-82-3604228928.py:151: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_name=, values=)
  worksheet.update('C22', [[f"{average_r2_disc_i:.4f}"]])
/tmp/ipython-input-82-3604228928.py:154: DeprecationWarning: The order of arguments in worksheet.update() has changed. Please pass values first and range_name secondor used named arguments (range_nam


Average R-squared scores:
IDim:  -5.637537598609924
PDim:  -2.8038182854652405
IDisc:  -12.904192543029785
PDisc:  -4.270830345153809
